# [aka.ms/pc-cng](https://aka.ms/pc-cng)

# Introduction

A *very brief* introduction to the Planetary Computer.

In [ ]:
import adlfs
import planetary_computer
import ipyleaflet
import requests
import shapely
from IPython.display import IFrame

The Planetary Computer hosts lots of geospatial data. Anyone can use it (ideally from Azure!).

<img src="data-catalog.png" width="50%"/>

Check out https://planetarycomputer.microsoft.com/catalog for the full catalog. We'll talk about how to access it in a bit.

We also provide APIs for searching and working with that data. For example, our STAC API:

In [ ]:
IFrame(
    "https://planetarycomputer-staging.microsoft.com/api/stac/v1/docs",
    width=800,
    height=400,
)

Finally, we offer some compute next to the data. Azure has *many* ways of doing compute (plain VMs, Azure Functions, Kubernets, Azure Batch, ...). We host a very convenient [Dask-enabled JupyterHub](https://planetarycomputer.microsoft.com/docs/overview/environment/). Right now we're on a similar JupyterHub deployment.

So that's the Planetary Computer: Geosptial Data + APIs + Compute, all on Azure.

## Cloud-native Principals

The Planetary Computer implements many cloud-native concepts. Here, we'll just list some of them.


1. You have *direct* access to *all* of the data
    - You have access to PBs of data
    - Data assets are hosted in the highly scalable Azure Blob Storage
    - You have direct access to the files, using plain HTTPs or Azure Blob Storage APIs. This means you can open the files using any tool that can speak HTTP
2. Cloud-native formats
    - Wherever possible, we use cloud-native / friendly file formats. We'll see examples using COG, Zarr, (geo)parquet, and COPC
3. Compute is next to the data
    - All of our files are in the West Europe Azure data region. For best performance, compute should be in that same data center.
4. Ability to scale
    - Azure makes it easy to get lots of compute


### Compute → Data

Putting the compute next to the data can be crucial for performance. Let's consider the simple task of reading the metadata from a COG file with `gdalinfo`.

Running this command from my laptop in Des Moines, IA, we spend a *lot* of time waiting:

```console
$ time gdalinfo /vsicurl/https://naipeuwest.blob.core.windows.net/naip/v002/ia/2019/ia_60cm_2019/42091/m_4209150_sw_15_060_20190828.tif > /dev/null
real    0m7.158s
user    0m0.195s
sys     0m0.032s
```

Running that from this Jupyter kernel, which is in the same Azure data center as the dataset, things look different.

In [ ]:
!time gdalinfo /vsicurl/https://naipeuwest.blob.core.windows.net/naip/v002/ia/2019/ia_60cm_2019/42091/m_4209150_sw_15_060_20190828.tif > /dev/null

So a nice 35x speedup!

## STAC

Having access to the data is great, but it's not enough. For example, how would you find all the Sentinel-2 images over Wyoming for July 2021? Consider what we'd do if we just had files in blob storage. We'll use `adlfs` to list some folders, to try to figure out the naming convention (we could also read the docs, but where's the fun in that?)

In [ ]:
token = planetary_computer.sas.get_token("sentinel2l2a01", "sentinel2-l2").token

fs = adlfs.AzureBlobFileSystem("sentinel2l2a01", credential=token)
fs.ls("sentinel2-l2/01/C/CV/2021")  # ...?

Some of those kinda look like dates. I don't know what the "C" and "DH" mean.

But I don't need to figure that out! STAC makes this kind of spatio-temporal filtering straightforward.

In [ ]:
import pystac_client

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

In [ ]:
%%time

wyoming_bbox = [-111.0717, 41.0296, -103.9965, 45.02695]
search = catalog.search(
    collections=["sentinel-2-l2a"], bbox=wyoming_bbox, datetime="2021-07-01/2021-07-31"
)
items = search.get_all_items()

In [ ]:
len(items)

Even better: STAC is a standard. It isn't specific to Sentinel-2, or even remote sensing data. Landsat Collection 2 Level-2, which uses a completely different folder structure in blob storage, can be searched by just chagning the collection ID.

In [ ]:
%%time

search = catalog.search(
    collections=["landsat-8-c2-l2"], bbox=wyoming_bbox, datetime="2021-07-01/2021-07-31"
)
landsat_items = search.get_all_items()
len(landsat_items)

STAC items are GeoJSON Features. So even though they're just metadata, we can treat them a bit like data. We can do things like plot the various tiles on a map.

In [ ]:
import geopandas

df = geopandas.GeoDataFrame.from_features(items.to_dict()).set_crs(4326)

df[["geometry", "s2:mgrs_tile", "datetime"]].explore(
    column="datetime", style_kwds={"fillOpacity": 0.1}, cmap="viridis"
)

## Data APIs

The Planetary Computer also provides a data API, based on [TiTiler](https://developmentseed.org/titiler/), which provides endpoints for some common geospatial analysis routines. This can be a nice alternative to setting up your own compute in Azure if you're doing something basic, like putting an image on a Map (or even more advanced things like mosaicing many images).

In [ ]:
def plot(item, map_kwargs={}):
    (tiles_url,) = requests.get(item.assets["tilejson"].href).json()["tiles"]
    center = shapely.geometry.shape(item.geometry).centroid.bounds[1::-1]

    m = ipyleaflet.Map(
        center=center, controls=[ipyleaflet.FullScreenControl()], **map_kwargs
    )
    m.add_layer(ipyleaflet.TileLayer(url=tiles_url))
    m.scroll_wheel_zoom = True
    return m

In [ ]:
plot(items[1])

Whoops, that's a pretty cloudy image. But STAC makes it easy to filter those out.

In [ ]:
wyoming_bbox = [-111.0717, 41.0296, -103.9965, 45.02695]
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=wyoming_bbox,
    datetime="2021-07-01/2021-07-31",
    query={"eo:cloud_cover": {"lt": 10}},
)
%time items = search.get_all_items()
len(items)

In [ ]:
plot(items[1], map_kwargs=dict(zoom=9))

Fun fact: the STAC and Data APIs power our [explorer](https://planetarycomputer.microsoft.com/explore?c=118.8189%2C37.4070&z=11.00).

### Scaling

We aren't going to see it today, but our Hub includes a Dask Gateway. This lets you create Dask clusters to distribute your computation, without having to worry about intrastructure things like Kubernetes.

```python
>>> from dask_gateway import GatewayCluster
>>> cluater = GatewayCluster()
>>> cluster.scale(40)
>>> client = cluster.get_client()
```

We'll be using Dask later to parallelize some computations, but just on a single machine instead of a cluster.